In [265]:
from cdpipelines import general
reload(general)
from cdpipelines import wgs_hla_typing
reload(wgs_hla_typing)
import pandas as pd
import os
import subprocess

output_dir = '/home/joreyna/trash/'

In [256]:
class WGS_HLAJobScript_extended(wgs_hla_typing.WGS_HLAJobScript):
    
    def select_vbseq_alleles(self, raw_vbseq, top_selections, hla_bed, hla_fasta):
        """
        raw_vbseq: str
            Path to the raw vbseq results (*.vbseq.avgdp')
            
        top_selection: list
            Indexes for selecting some allele based on the descent order.
            
        hla_bed: str
            Bed file with the coordinates of all HLA types. 
        
        hla_fasta: str
            Fasta file with the sequences of all HLA types.
            
        """
        
        # DETERMINING the top allele for each HLA gene.
        raw_vbseq = pd.read_table(raw_vbseq, header=None, names=['allele', 'mean_read_depth'])
        raw_vbseq['gene'] = [x[0] for x in raw_vbseq.allele.str.split('*')]
        gene_grps = raw_vbseq.groupby('gene')
        select_alleles = gene_grps.apply(\
              lambda x: x.sort_values('mean_read_depth', ascending=False).iloc[top_selections])
        select_alleles.reset_index(drop=True, inplace=True)
        select_alleles.set_index('allele', inplace=True)

        # PUllING the coordinates for the alleles of interest.  
        allele_coords = pd.read_table(hla_bed, names = ['accession', 'start', 'end', 'allele'], header=None)
        allele_coords = allele_coords[allele_coords['allele'].isin(select_alleles.index.tolist())]
        
        # SAVING the coordinates 
        output_bed = os.path.join(self.outdir, '{}_select_alleles.bed'.format(self.sample_name))
        allele_coords.to_csv(output_bed, index=False, header=False, sep='\t')
        
        # SAVING a version with reversed accession and allele locations
        rev_output_bed = os.path.join(self.outdir, '{}_rev_select_alleles.bed'.format(self.sample_name))
        rev_allele_coords = allele_coords[[ 'allele', 'start', 'end', 'accession']]
        rev_allele_coords.to_csv(rev_output_bed, index=False, header=False, sep='\t')

        # GENERATING the fasta file 
        output_fasta = os.path.join(self.outdir, '{}_select_alleles.fasta'.format(self.sample_name))
        cmd = 'bedtools getfasta -fi {} -bed {} -fo {} -name'.format(hla_fasta, output_bed, output_fasta)
        
        lines = self._add_execution_date(cmd)
        with open(self.filename, "a") as f:
            lines = '\n'.join(lines)
            f.write(lines)
        
        return output_bed, rev_output_bed, output_fasta 
    
    def bwa_map(
        self,
        ref,
        r1_fastq,
        r2_fastq,
        stringent=False,
        suffix=None,
    ):
        """
        Aligning paired end data to specific allele reference sequences using bwa.

        - bwa P is used for paired-end data.
        - samtools -F 4 is used to extract mapped reads only.


        Parameters
        __________
        hla_ref : str
            Path to hla_ref file.
        r1_fastq : str
            Path to input r1 fastq file.
        r2_fastq : str
            Path to input r2 fastq file.
            Path to bowtie2.

        Returns
        -------
        out_bam : str
            Path to bam file.
        """
        if suffix: 
            out_bam = os.path.join(self.outdir, '{}_{}.bam'.format(self.sample_name, suffix))
        else: 
            out_bam = os.path.join(self.outdir, '{}.bam'.format(self.sample_name))

        if stringent:
            cmd = 'bwa mem -P -B 40 -O 60 -E 10 -L 10000 -t {} {} {} {} | samtools view -F 4 -b - > {}'.\
                format(self.threads,
                ref,
                r1_fastq,
                r2_fastq,
                out_bam)
        else:
            cmd = 'bwa mem -P -L 10000 -t {} {} {} {} | samtools view -F 4 -b - > {}'.\
                format(self.threads,
                ref,
                r1_fastq,
                r2_fastq,
                out_bam)

        lines = self._add_execution_date(cmd)
        with open(self.filename, 'a') as f:
            f.write('\n'.join(lines))
        return out_bam

## Creating a sample job

In [257]:
pipe_dir = os.path.join(output_dir, 'align_first_allele')
sample_name = 'c17006e1-8649-4e9e-ba4f-7bd4ee752244_ds100'
sample = os.path.join(pipe_dir, sample_name)
job_suffix = 'testing'
threads = 1
memory = 4
queue = None
conda_env = 'hla'
modules = 'bwa,samtools'
wait_for = None

In [ ]:
raw_vbseq = '/projects/CARDIPS/pipeline/HLA_Typing/sample/WGS_Main/{0}/hla/{0}.vbseq.avgdp'.format(sample_name)
top_choices = [1]
hla_bed = '/repos/cardips-pipelines/HLA_Typing/sources/IPD_IMGT_HLA_Release_3.29.0_2017_07_27/hla_gen.fasta.bed'
hla_fasta = '/repos/cardips-pipelines/HLA_Typing/sources/IPD_IMGT_HLA_Release_3.29.0_2017_07_27/hla_gen.fasta'

r1_fastq = '/projects/CARDIPS/pipeline/HLA_Typing/sample/WGS_Main/{0}/reads/{0}_R1.fastq'.format(sample_name)
r2_fastq = '/projects/CARDIPS/pipeline/HLA_Typing/sample/WGS_Main/{0}/reads/{0}_R2.fastq'.format(sample_name)

In [ ]:
## Selecting alleles 
job = WGS_HLAJobScript_extended(sample_name, 'select_alleles', os.path.join(sample, 'qc'), 1, 1, 
                                queue = queue, 
                                conda_env=conda_env,
                                modules = 'bedtools,bwa')
select_alleles_job = job.jobname
job.add_input_file(raw_vbseq)
job.add_input_file(hla_bed)
job.add_input_file(hla_fasta)
select_alleles_bed, rev_select_alleles_bed, select_alleles_fasta = \
        job.select_vbseq_alleles(raw_vbseq, top_choices, hla_bed, hla_fasta)
select_alleles_bwa_indexes = job.bwa_index(select_alleles_fasta)
job.add_output_file(output_bed)
job.add_output_file(output_fasta)
job.write_end()
if not job.delete_sh:
    submit_commands.append(job.sge_submit_command())

## Aligning reads reference 
job = WGS_HLAJobScript_extended(sample_name, 'align_to_select_alleles', os.path.join(sample, 'qc'), 1, 4, 
                                queue = queue, 
                                conda_env=conda_env,
                                modules = 'bwa,samtools,sambamba',
                                wait_for = [select_alleles_job])
aln_select_jobname = job.jobname

job.add_input_file(r1_fastq)
job.add_input_file(r2_fastq)
select_alleles_bam = job.bwa_map(select_alleles_fasta, r1_fastq, r2_fastq, suffix='select_alleles')
select_alleles_sorted_bam = job.sambamba_sort(select_alleles_bam, suffix='select_alleles')
select_alleles_sorted_bai = job.sambamba_index(select_alleles_sorted_bam, suffix='select_alleles')

job.add_output_file(select_alleles_bam)
job.add_output_file(select_alleles_sorted_bam)
job.add_output_file(select_alleles_sorted_bai)
job.write_end()
if not job.delete_sh:
    submit_commands.append(job.sge_submit_command())

## Running mpileup
job = WGS_HLAJobScript_extended(sample_name, 'mpileup_select_alleles', os.path.join(sample, 'qc'), 1, 4, 
                                queue = queue, 
                                conda_env=conda_env,
                                modules = 'samtools/1.4',
                                wait_for = [aln_select_jobname])
mpile_select_jobname = job.jobname
job.add_input_file(select_alleles_sorted_bam)
job.add_input_file(select_alleles_fasta)
job.add_input_file(hla_bed)
select_alleles_mpile, select_alleles_mpile_parsed = job.samtools_mpileup(select_alleles_sorted_bam, select_alleles_fasta, 
                                   rev_select_alleles_bed, suffix='select_alleles')
job.add_output_file(select_alleles_mpile)
job.write_end()
if not job.delete_sh:
    submit_commands.append(job.sge_submit_command())

## Analyzing 

In [299]:
table = pd.read_table(select_alleles_mpile_parsed)
table['total_a'] = table['A'] + table['a']
table['total_c'] = table['C'] + table['c']
table['total_g'] = table['G'] + table['g']
table['total_t'] = table['T'] + table['t']
table['total_snps']  = table[['A', 'C', 'G', 'T', 'a', 'c', 'g', 't']].sum(axis=1)
table['proportion_a']  = table['total_a'] * 100 / table['total_snps']
table['proportion_c']  = table['total_c'] * 100 / table['total_snps']
table['proportion_g']  = table['total_g'] * 100 / table['total_snps']
table['proportion_t']  = table['total_t'] * 100 / table['total_snps']
table = table[['chr', 'loc', 'ref', 'proportion_a', 'proportion_c', 'proportion_g', 'proportion_t']]

hets = []
for index, sr in table.iterrows():
    ref_proportion = sr['proportion_{}'.format(sr.ref.lower())]
    
    if ref_proportion >= 40 and ref_proportion <= 60:
        hets.append(True)
    else:
        hets.append(False)
table['hets'] = hets